# Generating offspring photos with ComfyUI

## 1. Set Up

In [ ]:
!git clone https://github.com/comfyanonymous/ComfyUI.git

In [ ]:
!pip install -r requirements.txt

In [ ]:
import sys
import os
from pathlib import Path

In [ ]:
# Define my specific ComfyUI path
COMFY_PATH = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI")

# Verify the path exists
if not COMFY_PATH.exists():
    raise Exception(f"ComfyUI path not found at {COMFY_PATH}")
else:
    print(f"ComfyUI directory found at {COMFY_PATH}")

# Add ComfyUI to system path
if str(COMFY_PATH) not in sys.path:
    sys.path.append(str(COMFY_PATH))
    print("Added ComfyUI to system path")

In [ ]:
import requests
from tqdm import tqdm

In [ ]:
# Function to download models
def download_model(url, save_path):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    save_path = Path(save_path)
    save_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(save_path, 'wb') as f, tqdm(
        desc=save_path.name,
        total=total_size,
        unit='iB',
        unit_scale=True
    ) as pbar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            pbar.update(size)

In [ ]:
# Create model directories
models_path = COMFY_PATH / "models"
checkpoints_path = models_path / "checkpoints"
controlnet_path = models_path / "controlnet"

In [ ]:
# Create directories and print status
for path in [models_path, checkpoints_path, controlnet_path]:
    path.mkdir(parents=True, exist_ok=True)
    print(f"Created/verified directory: {path}")

## Start the ComfyUI server

In [ ]:
import subprocess
import webbrowser
from time import sleep

In [ ]:
# Start ComfyUI server
server_process = subprocess.Popen(
    ['python', str(COMFY_PATH / 'main.py'), '--listen', '0.0.0.0', '--port', '8188'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait a moment for the server to start
sleep(5)

# Open the UI in default browser
webbrowser.open('http://localhost:8188')

print("ComfyUI server started! The interface should open in your browser.")
print("Keep this notebook running while you use ComfyUI.")

## 3. Setting up Paths

In [34]:
import json
import csv
from datetime import datetime
import shutil

In [35]:
# Create input directory for my photo
input_path = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input")
input_path.mkdir(parents=True, exist_ok=True)

In [36]:
# Paths dictionary
PATHS = {
    "donors": Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/comfyui_source_files"),
    "output": Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/outputs/generated_offsprings"),
    "my_photo": input_path / "cindy.jpg" 
}

In [37]:
# Verify paths
for name, path in PATHS.items():
    if path.exists():
        print(f"✓ Found {name} directory: {path}")
    else:
        if name == "my_photo":
            print(f"\nPlease save your headshot photo as: {path}")
            print("Once saved, run this cell again to verify.")
        elif name == "output":
            path.mkdir(parents=True, exist_ok=True)
            print(f"Created output directory: {path}")
        else:
            print(f"❌ ERROR: {name} directory not found: {path}")

✓ Found donors directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/comfyui_source_files
✓ Found output directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/outputs/generated_offsprings
✓ Found my_photo directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/cindy.jpg


In [38]:
# Clean up previous output directories if necessary
project_outputs = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/thesis_outputs")
if project_outputs.exists():
    if project_outputs.is_symlink():
        project_outputs.unlink()
    elif project_outputs.is_dir():
        shutil.rmtree(project_outputs)

In [39]:
# Set up fresh directory structure in ComfyUI's output
comfy_output = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output")
donor_output = comfy_output / "donor_logs" / "aug_gpt_4_gem_wo" / "v1"
donor_output.mkdir(parents=True, exist_ok=True)

In [40]:
print("Cleaned up previous setup and created fresh directory structure.")
print(f"ComfyUI output directory: {comfy_output}")
print(f"Donor output directory: {donor_output}")

Cleaned up previous setup and created fresh directory structure.
ComfyUI output directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output
Donor output directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/donor_logs/aug_gpt_4_gem_wo/v1


## Batch Processing
Testing with 5 donors first to ensure everything works properly

In [41]:
# Import and Setup
import time
import random


In [42]:
# Update paths for batch processing
COMFY_OUTPUT = COMFY_PATH / "output"
DONORS_PATH = PATHS["donors"]
MY_PHOTO = PATHS["my_photo"]
BASE_WORKFLOW_PATH = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/user/default/workflows/base_workflow.json")

In [43]:
test_donors = sorted(list(DONORS_PATH.glob("*.jpeg")))[:5]
print("\nWill process these donors:")
for i, donor in enumerate(test_donors, 1):
    print(f"{i}. {donor.name}")


Will process these donors:
1. aug_cla_2_gem_wo.jpeg
2. aug_cla_3_gem_wo.jpeg
3. aug_cla_4_gem_wo.jpeg
4. aug_cla_5_gem_wo.jpeg
5. aug_cla_6_gem_demo.jpeg


In [44]:
# Utility Functions
def verify_saved_image(output_path: Path, donor_id: str, variation: int) -> bool:
    """Verify that image was saved and is valid"""
    expected_file = list(output_path.glob(f"{donor_id}_baby_{variation}*.png"))
    
    if not expected_file:
        print(f"❌ Warning: No output file found for {donor_id} variation {variation}")
        return False
        
    if expected_file[0].stat().st_size == 0:
        print(f"❌ Warning: Empty file generated for {donor_id} variation {variation}")
        return False
        
    print(f"✓ Successfully saved: {expected_file[0].name}")
    return True


In [45]:
def generate_random_seeds(count=3):
    """Generate list of random seeds for multiple generations"""
    return [random.randint(1, 2**31 - 1) for _ in range(count)]

In [46]:
# Load and modify base workflow
print(f"Loading base workflow from: {BASE_WORKFLOW_PATH}")
if not BASE_WORKFLOW_PATH.exists():
    raise FileNotFoundError(f"Base workflow not found at: {BASE_WORKFLOW_PATH}")

with open(BASE_WORKFLOW_PATH, 'r') as f:
    complete_workflow = json.load(f)
print("Successfully loaded base workflow")


Loading base workflow from: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/user/default/workflows/base_workflow.json
Successfully loaded base workflow


### Test Batch Processing Setup

In [54]:
def process_test_batch():
    total_donors = len(test_donors)

    # Define the output path for workflows
    workflow_output_path = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/workflows")

    for donor_idx, donor_path in enumerate(test_donors, 1):
        print(f"\n{'='*50}")
        print(f"Processing donor {donor_idx}/{total_donors}: {donor_path.name}")

        seeds = generate_random_seeds(3)

        for variation in range(1, 4):
            print(f"\n>> Starting variation {variation}/3")
            print("Generating random parameters...")

            # Create a copy of the complete workflow for this variation
            workflow = complete_workflow.copy()

            # Add version key to workflow if missing
            if "version" not in workflow:
                workflow["version"] = 1.0  # Set default version if not present

            # Update LoadImage nodes
            for node in workflow['nodes']:
                if node['type'] == 'LoadImage':
                    if node['id'] == 1:  # Your photo
                        node['widgets_values'] = [str(MY_PHOTO)]
                    elif node['id'] == 2:  # Donor photo
                        node['widgets_values'] = [str(donor_path)]

            # Add BlendImagesRandom node
            blend_node = {
                "id": 10 + variation,  # Unique ID for each variation
                "type": "BlendImagesRandom",
                "pos": [800, 400],  # Position in the UI (adjust as necessary)
                "size": [400, 100],
                "flags": {},
                "order": 4,
                "mode": 0,
                "properties": {
                    "inputs": [
                        {"name": "image1", "type": "IMAGE", "link": 5, "slot_index": 0},  # Link to first image
                        {"name": "image2", "type": "IMAGE", "link": 6, "slot_index": 0}   # Link to second image
                    ],
                    "outputs": [
                        {"name": "blended_image", "type": "IMAGE", "links": [11], "slot_index": 0}
                    ]
                }
            }
            workflow['nodes'].append(blend_node)

            # Update SaveImage node to save blended images
            for node in workflow['nodes']:
                if node['type'] == 'SaveImage':
                    donor_id = Path(donor_path).stem
                    node['widgets_values'] = [f"{donor_id}_baby_{variation}"]  # Dynamic filename

            # Save modified workflow for this variation
            workflow_path = workflow_output_path / f"{donor_path.stem}_v{variation}.json"
            workflow_path.parent.mkdir(parents=True, exist_ok=True)

            with open(workflow_path, 'w') as f:
                json.dump(workflow, f, indent=2)

            print(f"✓ Saved workflow to: {workflow_path}")



### Process Test Batch

In [55]:
# Starting the test batch
print("Starting test batch with randomized parameters for 5 donors...")
proceed = input("Press Enter to continue or Ctrl+C to cancel...")
process_test_batch()


Starting test batch with randomized parameters for 5 donors...

Processing donor 1/5: aug_cla_2_gem_wo.jpeg

>> Starting variation 1/3
Generating random parameters...
✓ Saved workflow to: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/workflows/aug_cla_2_gem_wo_v1.json

>> Starting variation 2/3
Generating random parameters...
✓ Saved workflow to: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/workflows/aug_cla_2_gem_wo_v2.json

>> Starting variation 3/3
Generating random parameters...
✓ Saved workflow to: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/workflows/aug_cla_2_gem_wo_v3.json

Processing donor 2/5: aug_cla_3_gem_wo.jpeg

>> Starting variation 1/3
Generating random parameters...
✓ Saved workflow to: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/workflows/aug_cla_3_gem_wo_v1.json

>> Starting variation 2/3
Generating rando